In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import nltk
import string

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
con = sqlite3.connect('database.sqlite')

In [3]:
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score !=3 LIMIT 5000""", con)

def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

In [4]:
actualScore = filtered_data['Score']     #choose score column from the filtered_data and store into actual data

positiveNegative = actualScore.map(partition)     #apply function over that actualdata  partition function


filtered_data['Score'] = positiveNegative        #coverting score 1,2,4,5 in to positive and negative reviews and again out into
#filtered_data


In [5]:
#print(filtered_data.head(2))
print(filtered_data.shape)
filtered_data.head(3)

(5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [13]:
#cleaning duplication reviews from the data ists important part   1st case same review and 2nd case in hepfulnessNumertor 
#is greater than the helpfulnessDenominator




In [6]:
#just to show this type of data is may doing duplicates

display =  pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score !=3 AND UserId = "AR5J8UI46CURR" ORDER BY ProductId""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [7]:
#sorting the data on the basis of productid

sorted_data = filtered_data.sort_values('ProductId', axis =0, ascending = True)

In [8]:
#cleaning duplicates data  which having sane userid profile name time and text

final = sorted_data.drop_duplicates(subset={"UserId","ProfileName", "Time","Text"}, keep='first',inplace=False)
final.shape

(4986, 10)

In [9]:
#finding % how much data remain 

(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

99.72

In [10]:
#2nd type remove duplications data like helpfulnessNumerator is less than helfulnessDenominator, which is impossible 

In [10]:
display =  pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score !=3 AND Id=44737 OR Id =64422 ORDER BY ProductID""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [11]:
final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [12]:
print(final.shape)

(4986, 10)


In [13]:
# counts how much review are positive and how much negative

final['Score'].value_counts()

positive    4178
negative     808
Name: Score, dtype: int64

In [14]:
final['Text'].values[0]

'Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.'

In [15]:
#now using technique to convert text in to vector .....before it remove stopwords and using porterStemmer

#bow using to convert text into vector

In [16]:
count_vector = CountVectorizer()
final_counts = count_vector.fit_transform(final['Text'].values) 

In [17]:
#bow, tf-idf,word2vec, avg word2vec  etc

In [18]:
final_counts.shape  #shape of that sparse matrix

(4986, 13510)

In [19]:
import re
i = 0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break;
    i+=1;

0
Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [20]:
import re
import string

In [21]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [22]:
stop = set(stopwords.words('english'))
stop.remove('not')
stop.remove('nor')
stop.remove('down')
stop.remove('no')
print(stop)

{'an', 'her', 'to', 's', 'a', 'just', 'all', 'does', 'up', 'when', 'above', 'wasn', 'as', 'him', 'theirs', 'what', 'haven', 'aren', 'between', 'is', 'there', "you've", 'don', "mightn't", 'ours', "wouldn't", 'were', "mustn't", 'that', 'hasn', 'was', 'any', "haven't", "won't", 'then', 'having', 'only', 'so', 'me', 'she', 'but', 'some', 'y', 'after', 'during', 'myself', 'himself', 'hers', 'them', "aren't", 'few', 'didn', 'our', 'are', "you'll", 'they', 'for', 'am', 'his', 'more', 'too', 'than', 'under', "weren't", 'this', 'being', 'ourselves', "couldn't", 't', "hadn't", 'until', 'shan', 'm', 'had', 'weren', 'themselves', 'here', 'o', 'on', "doesn't", 'before', 'hadn', 'if', 'by', 'wouldn', 'it', 'at', 'why', 'won', 'been', 'other', 'doesn', "should've", 'isn', 'yours', 'whom', 'yourself', 'such', 'should', 'itself', 'those', "that'll", 'of', 'my', 'yourselves', 'from', 'd', 'did', 'ain', 'in', "shan't", "shouldn't", 'has', "you'd", 'how', 're', "hasn't", 'i', 'or', 'over', 'will', 'each',

In [23]:
#stemming of word tasty

sno = nltk.stem.SnowballStemmer('english')
print(sno.stem('tasty'))
print(sno.stem('delicious'))

tasti
delici


In [24]:
def denocrate(phase):
    phase = re.sub(r"\'t"," not ",phase)
    phase = re.sub(r"\'ll", " will ",phase)
    phase = re.sub(r"\'re" , " are " , phase)
    phase = re.sub(r"\'d" , " would " , phase)
    phase = re.sub(r"\'m" , " am " , phase)
    phase = re.sub(r"\'ve" , " have " , phase)
    phase = re.sub(r"\'s" , " is " , phase)
    return phase

In [25]:
cleaned_text = []
for text in tqdm(final["Text"]):
    soup = BeautifulSoup(text , "lxml")        ###### remove html tags
    text = soup.get_text()
    text = re.sub(r"http\S+", "", text)           #### remove links 
    text = denocrate(text)
    text = re.sub("\S*\d\S*", "", text).strip()     #### removes wods which have deciml numbers
    text = re.sub('[^A-Za-z]+', " ", text).strip()   ####  removes special characters
   ## text = ' '.join(word.lower() for word in text.split() if word.lower() not in stop_words)
    ## text = ' '.join(stem.stem(word) for word in text.split() if len(word)>2)
    cleaned_text.append(text)

100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:01<00:00, 2645.59it/s]


In [26]:
type(cleaned_text)

list

In [44]:
cleaned_text[0]

'Why is this when the same product is available for here Victor and traps are unreal of course total fly genocide Pretty stinky but only right nearby'

In [45]:
final["Cleaned_text"] = cleaned_text

In [46]:
final.shape

(4986, 11)

In [47]:
final["Cleaned_text"][0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality The product looks more like a stew than a processed meat and it smells better My Labrador is finicky and she appreciates this product better than most'

In [48]:
### now we have a problem that each time we want to use these vectors we have to run ths whole file so we either we could store
### the vector converted form of these cleaned texts to excel or database but this is too time consuming so i will store this 
##3 this cleaned text to new database

In [49]:
#store final table into SQlite table for future
conn = sqlite3.connect("final.sqlite")

In [50]:
final.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Cleaned_text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,positive,1282953600,thirty bucks?,Why is this $[...] when the same product is av...,Why is this when the same product is available...


In [51]:
cursor_object = conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn ,schema = None ,if_exists='replace' ,index = False )

In [65]:
 #>>>>>>>>>>>>>>>>>>>>>>>>###   >>>>>>     now it is time for featurisation(vectorisation of texts) 1st BOW

In [52]:
count_vector = CountVectorizer()
final_counts = count_vector.fit_transform(final['Text'].values)

In [53]:
type(count_vector)

sklearn.feature_extraction.text.CountVectorizer

In [33]:
type(final_counts)     #it gives sparse matrix which containing lots zeroes

scipy.sparse.csr.csr_matrix

In [34]:
final_counts.shape  #shape of sparse matrix

(4986, 13510)

In [40]:
count_vect = CountVectorizer(ngram_range=(1,2))
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [41]:
final_bigram_counts.shape

(4986, 148211)

In [54]:
count = CountVectorizer()
count.fit(final["Cleaned_text"])
bow_vector = count.transform(final["Cleaned_text"])
bow_vector.shape

(4986, 13237)

In [55]:
type(bow_vector)

scipy.sparse.csr.csr_matrix

In [56]:
### trying to store BOW vector into a new excel file


bow_vector2 = bow_vector.toarray()
bow_df = pd.DataFrame(data=bow_vector2)
bow_df.columns = np.arange(13237)
bow_df.index = np.arange(4986)

In [57]:
bow_df["Score"] = final["Score"]

In [58]:
bow_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,13228,13229,13230,13231,13232,13233,13234,13235,13236,Score
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positive
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negative
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positive
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negative
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positive


In [59]:
###  writer = pd.ExcelWriter("bow.xlsx")
###  bow_df.to_excel(writer )dd

In [60]:
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>#tf-idf
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>tf-idf


tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [129]:
type(tf_idf_vect)
type(final_tf_idf)

scipy.sparse.csr.csr_matrix

In [61]:
final_tf_idf.shape

(4986, 148211)

In [62]:
#to get the features name we use .get_feature_names()

features_tf_idf = tf_idf_vect.get_feature_names()

In [63]:
len(features_tf_idf)

148211

In [118]:
features_tf_idf[4000:4005]

['agree', 'agree and', 'agree it', 'agree love', 'agree mark']

In [64]:
#convert a row into row sparsematrix to numpy array
#to get a one row

print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [65]:

def get_top_tf_idf_words(row,features_tf_idf, top_n=25):
    top_ids = np.argsort(row)[::-1][: top_n]
    top_feats= [(features_tf_idf[i],row[i]) for i in top_ids]
    df = pd.DataFrame(top_feats)
    df.columns = [ 'feature', 'tfidf']
    return df
top_tfidf = get_top_tf_idf_words(final_tf_idf[1,:].toarray()[0],features_tf_idf,25)


In [66]:
print(top_tfidf)

          feature     tfidf
0        fly bait  0.274736
1     seasons can  0.274736
2     for seasons  0.274736
3          victor  0.262108
4        bait for  0.262108
5      the victor  0.262108
6      victor fly  0.262108
7             fly  0.246199
8         seasons  0.246199
9            bait  0.240521
10        beat it  0.235720
11       can beat  0.201504
12       used the  0.192545
13      have used  0.186498
14           beat  0.173265
15       it great  0.169864
16  great product  0.154497
17        we have  0.142644
18           used  0.109802
19             we  0.092118
20        product  0.080693
21            can  0.077881
22          great  0.074089
23           have  0.061395
24            for  0.051863


In [ ]:
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>word2vec
>>>>>>>>>>>>>>>>>>word2vec